In [2]:
import pandas as pd
import numpy as np
import nltk
import re 
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


In [3]:
data = pd.read_csv("Text_Data.csv")
data

,Text,Emotion
0,"During the period of falling in love, each tim...",joy
1,When I was involved in a traffic accident.,fear
2,When I was driving home after several days of...,anger
3,When I lost the person who meant the most to me.,sadness
4,The time I knocked a deer down - the sight of ...,disgust
...,...,...
7475,Two years back someone invited me to be the tu...,anger
7476,I had taken the responsibility to do something...,sadness
7477,I was at home and I heard a loud sound of spit...,disgust
7478,I did not do the homework that the teacher had...,shame


In [4]:
import nltk
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
#Object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def text_process(text):
    no_punc = [char for char in text if char not in string.punctuation]
    no_punc = "".join(no_punc)
    no_punc = [lemmatizer.lemmatize(word) for word in no_punc.split() if word not in stopwords.words("english")]
    no_punc  = " ".join(no_punc)
    return no_punc
data["Text2"] = data["Text"].apply(text_process)


In [6]:
# data.to_csv("ProcessedData.csv")

In [5]:
data.head()

,Text,Emotion,Text2
0,"During the period of falling in love, each tim...",joy,During period falling love time met especially...
1,When I was involved in a traffic accident.,fear,When I involved traffic accident
2,When I was driving home after several days of...,anger,When I driving home several day hard work moto...
3,When I lost the person who meant the most to me.,sadness,When I lost person meant
4,The time I knocked a deer down - the sight of ...,disgust,The time I knocked deer sight animal injury he...


In [7]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data["Text2"], data['Emotion'], test_size = 0.20, random_state = 121)

def train_test(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    train_acc = accuracy_score(y_train, clf.predict(X_train))
    test_acc = accuracy_score(y_test, clf.predict(X_test))
    return train_acc, test_acc


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfVect = TfidfVectorizer()
X_train = tfVect.fit_transform(X_train)
X_test = tfVect.transform(X_test)

In [10]:
# with open("tfDict.pickle","wb") as dic:
#     pickle.dump(tfVect,dic)

<IPython.core.display.Javascript object>

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.calibration import CalibratedClassifierCV

svc = SVC(probability=True)
# lsvc = LinearSVC(random_state=123)
# lsvc = CalibratedClassifierCV(lsvc)
# rforest = RandomForestClassifier(random_state=123)
# mnb = MultinomialNB()
# dtree = DecisionTreeClassifier()

clifs = [svc]#,rforest, lsvc, rforest]# dtree]

# train and test them 
print("| {:25} | {} | {} |".format("Classifier", "Training Accuracy", "Test Accuracy"))
print("| {} | {} | {} |".format("-"*25, "-"*17, "-"*13))
for clf in clifs: 
    clf_name = clf.__class__.__name__
    train_acc, test_acc = train_test(clf, X_train, X_test, y_train, y_test)
    print("| {:25} | {:17.7f} | {:13.7f} |".format(clf_name, train_acc, test_acc))

| Classifier                | Training Accuracy | Test Accuracy |
| ------------------------- | ----------------- | ------------- |
| SVC                       |         0.9767714 |     0.5855615 |


In [551]:
# import pickle
# with open("TextEmotionSVC.h5" ,"wb") as model:
#     pickle.dump(clf,model)

In [12]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,clf.predict(X_test)))

              precision    recall  f1-score   support

       anger       0.44      0.54      0.49       232
     disgust       0.55      0.62      0.58       206
        fear       0.70      0.70      0.70       211
       guilt       0.47      0.50      0.48       202
         joy       0.73      0.71      0.72       214
     sadness       0.71      0.63      0.66       210
       shame       0.57      0.41      0.48       221

    accuracy                           0.59      1496
   macro avg       0.59      0.59      0.59      1496
weighted avg       0.59      0.59      0.59      1496



In [15]:
import pickle
import nltk
import string
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer

#Object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def text_process(text):
    no_punc = [char for char in text if char not in string.punctuation]
    no_punc = "".join(no_punc)
    no_punc = [lemmatizer.lemmatize(word) for word in no_punc.split() if word not in stopwords.words("english")]
    no_punc  = " ".join(no_punc)
    return no_punc
    
def textEmotions(text):
    with open("TextEmotionSVC.h5", 'rb') as model:
        clf = pickle.load(model)

    with open("tfDict.pickle", 'rb') as tfdict:
        tfVect = pickle.load(tfdict)
    
    output = []
    emoji_dict = {"joy":"😂", "fear":"😱", "anger":"😠", "sadness":"😢", "disgust":"😒", "shame":"😳", "guilt":"😳"}
    emotions = ["joy", 'fear', "anger", "sadness", "disgust", "shame", "guilt"]
    emotions.sort()

    features = tfVect.transform([text])
    prediction_prob = clf.predict_proba(features)[0]
    prediction = clf.predict(features)[0]
    #print(text)
    #print("-"*148)
    # print(emoji_dict[prediction], prediction)
    # print("-"*148)
    for i in range(len(prediction_prob)):
        #print(emotions[i],np.round(prediction_prob[i],3))        
        output.append([emotions[i],np.round(prediction_prob[i],3)])
    return(output)

In [4]:
text = """He gave the words of greeting, and the first regiment roared "Hurrah!" so deafeningly, continuously, 
and joyfully that the men themselves were awed by their multitude and the immensity of the power they constituted."""
text = text_process(text)
Emotionpred = textEmotions(text)

He gave word greeting first regiment roared Hurrah deafeningly continuously joyfully men awed multitude immensity power constituted
----------------------------------------------------------------------------------------------------------------------------------------------------
anger 0.058
disgust 0.144
fear 0.1
guilt 0.052
joy 0.524
sadness 0.065
shame 0.059
